# データストアの使用

以前のラボでは、Azure MLワークスペースに接続し、実験スクリプトが実行されているローカルCSVファイルのデータに基づいて簡単な実験を実行しました。ここで、クラウドに保存されたデータを操作します。

> **重要**: このノートブックのコードは、[Lab 4A](labdocs/Lab04A.md)の最初の2つのタスクを完了していることを前提としています。まだ行っていない場合は、今すぐ行ってください！

## ワークスペースに接続する

最初に行う必要があるのは、Azure ML SDKを使用してワークスペースに接続することです。

> **Note**: 前の演習を完了してから、Azureサブスクリプションとの認証済みセッションの有効期限が切れた場合、再認証するように求められます。

In [ ]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## ワークスペースでデータストアを表示する

ワークスペースには、[前のタスク](labdocs/Lab04A.md)で作成した**aml_data**データストアなど、いくつかのデータストアが含まれています。

次のコードを実行して*default*データストアを取得してから、どのデータストアがデフォルトであるかを示します。

In [ ]:
# Get the default datastore
default_ds = ws.get_default_datastore()

# Enumerate all datastores, indicating which is the default
for ds_name in ws.datastores:
    print(ds_name, "- Default =", ds_name == default_ds.name)

## 使用するデータストアを取得する

**aml_data**データストアを使用するため、名前で取得する必要があります:

In [ ]:
from azureml.core import Datastore

aml_datastore = Datastore.get(ws, 'aml_data')
print(aml_datastore.name,":", aml_datastore.datastore_type + " (" + aml_datastore.account_name + ")")

## デフォルトのデータストアを設定する

このコースでは、主に**aml_data**データストアを使用します。便宜上、デフォルトのデータストアになるように設定できます:

In [ ]:
ws.set_default_datastore('aml_data')
default_ds = ws.get_default_datastore()
print(default_ds.name)

## データストアへのデータのアップロード

作業するデータストアを特定したので、実験スクリプトが実際に実行されている場所に関係なく、ワークスペースで実行されている実験にアクセスできるように、ローカルファイルシステムからファイルをアップロードできます。

In [ ]:
default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'], # Upload the diabetes csv files in /data
                       target_path='diabetes-data/', # Put it in a folder path in the datastore
                       overwrite=True, # Replace existing files of the same name
                       show_progress=True)

## データストアからモデルをトレーニングする

上記のコードセルにファイルをアップロードすると、コードが*データ参照*を返したことに注意してください。データ参照は、スクリプトがデータストアの場所にあるデータにアクセスできるように、スクリプトの実行場所に関係なく、データストア内のフォルダーへのパスをスクリプトに渡す方法を提供します。

次のコードは、糖尿病CSVファイルをアップロードした**diabetes-data**フォルダーへの参照を取得し、特に*download*のデータ参照を構成します。つまり、フォルダーの内容をダウンロードするために使用できます。データ参照が使用されている計算コンテキストへ。データのダウンロードは、ローカルコンピューティングで処理される少量のデータに適しています。リモートコンピューティングを使用する場合、データストアの場所を*マウント*するようにデータ参照を構成し、データソースから直接データを読み取ることもできます。

> **詳しくは**: データストアの使用の詳細については、[Azure MLドキュメント](https://docs.microsoft.com/azure/machine-learning/how-to-access-data)を参照してください。

In [ ]:
data_ref = default_ds.path('diabetes-data').as_download(path_on_compute='diabetes_data')
print(data_ref)

トレーニングスクリプトでデータ参照を使用するには、そのパラメーターを定義する必要があります。次の2つのコードセルを実行して作成します:

1. **diabetes_training_from_datastore**という名前のフォルダー
2. 渡されたデータ参照パラメーターによって参照されるフォルダー内のすべてのCSVファイルのトレーニングデータを使用して、分類モデルをトレーニングするスクリプト。

In [ ]:
import os

# Create a folder for the experiment files
experiment_folder = 'diabetes_training_from_datastore'
os.makedirs(experiment_folder, exist_ok=True)
print(experiment_folder, 'folder created.')

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# Import libraries
import os
import argparse
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# Get parameters
parser = argparse.ArgumentParser()
parser.add_argument('--regularization', type=float, dest='reg_rate', default=0.01, help='regularization rate')
parser.add_argument('--data-folder', type=str, dest='data_folder', help='data folder reference')
args = parser.parse_args()
reg = args.reg_rate

# Get the experiment run context
run = Run.get_context()

# load the diabetes data from the data reference
data_folder = args.data_folder
print("Loading data from", data_folder)
# Load all files and concatenate their contents as a single dataframe
all_files = os.listdir(data_folder)
diabetes = pd.concat((pd.read_csv(os.path.join(data_folder,csv_file)) for csv_file in all_files))

# Separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

os.makedirs('outputs', exist_ok=True)
# note file saved in the outputs folder is automatically uploaded into experiment record
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

スクリプトは、パラメータとして渡されたデータ参照からトレーニングデータをロードするため、実験を実行するときにスクリプトパラメータを設定してファイル参照を渡すだけで済みます。

In [ ]:
from azureml.train.sklearn import SKLearn
from azureml.core import Experiment
from azureml.widgets import RunDetails

# Set up the parameters
script_params = {
    '--regularization': 0.1, # regularization rate
    '--data-folder': data_ref # data reference to download files from datastore
}


# Create an estimator
estimator = SKLearn(source_directory=experiment_folder,
                    entry_script='diabetes_training.py',
                    script_params=script_params,
                    compute_target = 'local'
                   )

# Create an experiment
experiment_name = 'diabetes-training'
experiment = Experiment(workspace = ws, name = experiment_name)

# Run the experiment
run = experiment.submit(config=estimator)
# Show the run details while running
RunDetails(run).show()
run.wait_for_completion()

実験を初めて実行するときは、Python環境のセットアップに時間がかかる場合があります-以降の実行はより高速になります。

実験が完了したら、ウィジェットで**azureml-logs/70_driver_log.txt**出力ログを表示して、データファイルがダウンロードされたことを確認します。

すべての実験と同様に、[Azure ML studio](https://ml.azure.com)で実験の詳細を表示し、生成されたメトリックとファイルを取得するコードを作成できます。:

In [ ]:
# Get logged metrics
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

ここでも、実験によって訓練されたモデルを登録できます。

In [ ]:
from azureml.core import Model

# Register the model
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'SKLearn Estimator (using Datastore)'}, properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

# List the registered models
print("Registered Models:")
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

この演習では、*datastore*の形式でデータを操作するためのいくつかのオプションを検討しました。

Azure Machine Learningは、*datasets*の形式でデータの抽象化をさらに強化します。これについては、次に説明します。